In [1]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.path.dirname(os.path.abspath("__file__"))).resolve()
parent, root = file.parent, file.parents[1]
sys.path.append(str(root))

import torch
from dtmgp.utils.sparse_grid.sgdesign import SparseGridDesign
from dtmgp.utils.sparse_grid.hyperbolic_cross import HyperbolicCrossDesign

In [2]:
# test Hyperbolic Cross Design
design_class = HyperbolicCrossDesign
deg = 3
input_bd = [0,1]

print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_bd=input_bd).points )
print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_bd=input_bd).lefts)
print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_bd=input_bd).rights )
print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_bd=input_bd).indices_sort )

tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750])
tensor([  -inf,   -inf, 0.5000,   -inf, 0.2500, 0.5000, 0.7500])
tensor([   inf, 0.5000,    inf, 0.2500, 0.5000, 0.7500,    inf])
tensor([3, 1, 5, 0, 2, 4, 6])


In [3]:
# initial setting
d = 2
eta = 4
input_bd = [[0,1]]*d # None
design_class = HyperbolicCrossDesign

# generate sparse grid design
sg = SparseGridDesign(d, eta, input_bd=input_bd, design_class=design_class).gen_sg(dyadic_sort=True, return_neighbors=True)
x_tot = sg.pts_tot
id_prt = sg.id_prt # use self.tot_pts[ id_x_prt[t_sum, prt], : ] to extract grid points in each smolyak iter
pts_prt = sg.pts_prt # use self.pts_prt[t_sum, prt] to extract a d-dimensional list, each entry is one-dim points forming the sgdesign
pts_prt_set = sg.pts_prt_set

pts_set = sg.pts_set
n_pts = sg.n_pts

design_str_prt = sg.design_str_prt


indices_prt = sg.indices_prt
indices_prt_set = sg.indices_prt_set

indices_tot = sg.indices_tot
indices_set = sg.indices_set

In [4]:
pts_set

tensor([[0.5000, 0.5000],
        [0.5000, 0.2500],
        [0.5000, 0.7500],
        [0.5000, 0.1250],
        [0.5000, 0.3750],
        [0.5000, 0.6250],
        [0.5000, 0.8750],
        [0.2500, 0.5000],
        [0.2500, 0.2500],
        [0.2500, 0.7500],
        [0.7500, 0.5000],
        [0.7500, 0.2500],
        [0.7500, 0.7500],
        [0.1250, 0.5000],
        [0.3750, 0.5000],
        [0.6250, 0.5000],
        [0.8750, 0.5000]], dtype=torch.float64)

In [5]:
design_str_prt[eta,2][0].points

tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750])

In [6]:
pts_prt[eta,2]

[tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750]),
 tensor([0.5000])]

In [7]:
indices_prt[eta,2]

[tensor([3, 1, 5, 0, 2, 4, 6]), tensor([3])]

In [8]:
print(indices_set)

indices_select = indices_prt_set[3,1]
print(indices_select)

tensor([[3, 3],
        [3, 1],
        [3, 5],
        [3, 0],
        [3, 2],
        [3, 4],
        [3, 6],
        [1, 3],
        [1, 1],
        [1, 5],
        [5, 3],
        [5, 1],
        [5, 5],
        [0, 3],
        [2, 3],
        [4, 3],
        [6, 3]])
tensor([[3, 3],
        [1, 3],
        [5, 3]])


In [9]:
list_full = [tuple(l) for l in indices_set.tolist()] # [n_u, d] size list of tuple
list_select = [tuple(l) for l in indices_select.tolist()] # [n_arrow, d] size list of tuple

index_dict = dict((value, idx) for idx,value in enumerate(list_full))
index_select = torch.tensor( [index_dict[x] for x in list_select] )
index_select

tensor([ 0,  7, 10])

In [10]:
i = torch.tensor([[0, 1, 2, 0],
                   [1, 1, 0, 2]])
v = torch.tensor([1, 2, 3, 4], dtype=float)
sp_test = torch.sparse_coo_tensor(i, v, [5, 5])
a = torch.randn((5,5), dtype=float)
print(a)
print(sp_test)
print(sp_test @ a)

tensor([[-0.4268, -0.6943,  1.1826, -0.2031, -1.6386],
        [-0.0306,  0.6142,  1.0506,  0.5527,  0.1001],
        [-0.0238,  0.2904, -0.6883,  1.5236,  0.1345],
        [-0.6331,  0.3622, -0.6110, -1.0555, -1.7768],
        [-1.0859,  0.3073,  0.6711,  0.4672, -1.1183]], dtype=torch.float64)
tensor(indices=tensor([[0, 1, 2, 0],
                       [1, 1, 0, 2]]),
       values=tensor([1., 2., 3., 4.]),
       size=(5, 5), nnz=4, dtype=torch.float64, layout=torch.sparse_coo)
tensor([[-0.1259,  1.7756, -1.7026,  6.6472,  0.6381],
        [-0.0612,  1.2283,  2.1011,  1.1054,  0.2001],
        [-1.2805, -2.0829,  3.5477, -0.6094, -4.9159],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], dtype=torch.float64)


In [11]:
sp_id = sp_test._indices().T
print(sp_id)
print(index_select)

tensor([[0, 1],
        [1, 1],
        [2, 0],
        [0, 2]])
tensor([ 0,  7, 10])


In [12]:
index_select[sp_id]

tensor([[ 0,  7],
        [ 7,  7],
        [10,  0],
        [ 0, 10]])

In [13]:
sp_test_new = torch.sparse_coo_tensor(index_select[sp_test._indices()], sp_test._values(), [17, 17])
sp_test_new

tensor(indices=tensor([[ 0,  7, 10,  0],
                       [ 7,  7,  0, 10]]),
       values=tensor([1., 2., 3., 4.]),
       size=(17, 17), nnz=4, dtype=torch.float64, layout=torch.sparse_coo)